### **SPARK STREAMING**

In [0]:
entities = ['customers', 'trips', 'vehicles', 'locations', 'payments', 'drivers']

In [0]:
for entity in entities:
    # Static read to infer schema
    df_batch = spark.read.format("csv") \
        .option("header", True) \
        .option("inferSchema", True) \
        .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")

    schema_entity = df_batch.schema

    # Streaming read using the inferred schema
    df_stream = spark.readStream.format("csv") \
        .option("header", True) \
        .schema(schema_entity) \
        .load(f"/Volumes/pysparkdbt/source/source_data/{entity}/")

    # Stream write to Delta
    df_stream.writeStream.format("delta") \
        .outputMode("append") \
        .option("checkpointLocation", f"/Volumes/pysparkdbt/bronze/checkpoint/{entity}") \
        .trigger(once=True) \
        .toTable(f"pysparkdbt.bronze.{entity}")
